In [2]:
import pandas as pd

In [3]:
text="""[
  {
    "title": "The Hobbit",
    "author": "J.R.R. Tolkien",
    "publisher": "George Allen & Unwin",
    "year": 1937
  },
  {
    "title": "The Fellowship of the Ring",
    "author": "J.R.R. Tolkien",
    "publisher": "George Allen & Unwin",
    "year": 1954
  },
  {
    "title": "Nineteen Eighty-Four",
    "author": "George Orwell",
    "publisher": "Secker & Warburg",
    "year": 1949
  },
  {
    "title": "Brave New World",
    "author": "Aldous Huxley",
    "publisher": "Chatto & Windus",
    "year": 1932
  },
  {
    "title": "The Catcher in the Rye",
    "author": "J.D. Salinger",
    "publisher": "Little, Brown and Company",
    "year": 1951
  }
]"""
with open('Data/books.json','w') as f:
    print(text,file=f)

In [5]:
df = pd.read_json('Data/books.json')
df

,title,author,publisher,year
0,The Hobbit,J.R.R. Tolkien,George Allen & Unwin,1937
1,The Fellowship of the Ring,J.R.R. Tolkien,George Allen & Unwin,1954
2,Nineteen Eighty-Four,George Orwell,Secker & Warburg,1949
3,Brave New World,Aldous Huxley,Chatto & Windus,1932
4,The Catcher in the Rye,J.D. Salinger,"Little, Brown and Company",1951


In [ ]:
text = """title,author,publisher,year
The Hobbit,J.R.R. Tolkien,George Allen & Unwin,1937
The Fellowship of the Ring,J.R.R. Tolkien,George Allen & Unwin,1954
Nineteen Eighty-Four,George Orwell,Secker & Warburg,1949
Brave New World,Aldous Huxley,Chatto & Windus,1932
The Catcher in the Rye,J.D. Salinger,"Little, Brown and Company",1951"""
with open('Data/books.csv','w') as f:
    print(text,file=f)

In [2]:
df = pd.read_csv('Data/books.csv')

In [3]:
df

,title,author,publisher,year
0,The Hobbit,J.R.R. Tolkien,George Allen & Unwin,1937
1,The Fellowship of the Ring,J.R.R. Tolkien,George Allen & Unwin,1954
2,Nineteen Eighty-Four,George Orwell,Secker & Warburg,1949
3,Brave New World,Aldous Huxley,Chatto & Windus,1932
4,The Catcher in the Rye,J.D. Salinger,"Little, Brown and Company",1951


In [4]:
df.to_parquet("Data/books.parquet", engine="fastparquet", index=False)

In [5]:
df_parquet = pd.read_parquet("Data/books.parquet", engine="fastparquet")

In [6]:
df_parquet

,title,author,publisher,year
0,The Hobbit,J.R.R. Tolkien,George Allen & Unwin,1937
1,The Fellowship of the Ring,J.R.R. Tolkien,George Allen & Unwin,1954
2,Nineteen Eighty-Four,George Orwell,Secker & Warburg,1949
3,Brave New World,Aldous Huxley,Chatto & Windus,1932
4,The Catcher in the Rye,J.D. Salinger,"Little, Brown and Company",1951


In [7]:
import sqlite3

In [8]:
conn = sqlite3.connect("Data/books.db")

In [10]:
cur = conn.cursor()


In [11]:
# Create tables
cur.execute("""
CREATE TABLE IF NOT EXISTS publishers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year INTEGER,
    publisher_id INTEGER,
    FOREIGN KEY (publisher_id) REFERENCES publishers(id)
)
""")

# Insert a publisher
cur.execute("INSERT INTO publishers (name) VALUES (?)", ("George Allen & Unwin",))
publisher_id = cur.lastrowid

# Insert a book linked to that publisher
cur.execute("INSERT INTO books (title, author, year, publisher_id) VALUES (?, ?, ?, ?)",
            ("The Hobbit", "J.R.R. Tolkien", 1937, publisher_id))
conn.commit()


In [12]:
# Query join
cur.execute("""
SELECT b.title, b.author, p.name AS publisher
FROM books b
JOIN publishers p ON b.publisher_id = p.id
""")
print(cur.fetchall())

conn.close()

[('The Hobbit', 'J.R.R. Tolkien', 'George Allen & Unwin')]


In [13]:
from pymongo import MongoClient

In [14]:
client = MongoClient("mongodb://localhost:27017/")

In [15]:
db = client["library"]

In [16]:
publishers = db["publishers"]
books = db["books"]

In [17]:
publishers.delete_many({})
books.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [18]:
# 5. Insert publishers
pub_allen = publishers.insert_one({"name": "George Allen & Unwin"}).inserted_id
pub_secker = publishers.insert_one({"name": "Secker & Warburg"}).inserted_id
pub_chatto = publishers.insert_one({"name": "Chatto & Windus"}).inserted_id
pub_little = publishers.insert_one({"name": "Little, Brown and Company"}).inserted_id

# 6. Insert books with publisher references
books.insert_many([
    {"title": "The Hobbit", "author": "J.R.R. Tolkien", "year": 1937, "publisher_id": pub_allen},
    {"title": "The Fellowship of the Ring", "author": "J.R.R. Tolkien", "year": 1954, "publisher_id": pub_allen},
    {"title": "Nineteen Eighty-Four", "author": "George Orwell", "year": 1949, "publisher_id": pub_secker},
    {"title": "Brave New World", "author": "Aldous Huxley", "year": 1932, "publisher_id": pub_chatto},
    {"title": "The Catcher in the Rye", "author": "J.D. Salinger", "year": 1951, "publisher_id": pub_little},
])

# 7. Query: find all books and join publisher info manually
for book in books.find():
    publisher = publishers.find_one({"_id": book["publisher_id"]})
    print(f"{book['title']} by {book['author']} "
          f"was published by {publisher['name']} in {book['year']}.")

The Hobbit by J.R.R. Tolkien was published by George Allen & Unwin in 1937.
The Fellowship of the Ring by J.R.R. Tolkien was published by George Allen & Unwin in 1954.
Nineteen Eighty-Four by George Orwell was published by Secker & Warburg in 1949.
Brave New World by Aldous Huxley was published by Chatto & Windus in 1932.
The Catcher in the Rye by J.D. Salinger was published by Little, Brown and Company in 1951.
